In [1]:
import tensorflow as tf
import random
import matplotlib.pyplot as plt

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/" , one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
def xavier_init(n_inputs,n_outputs, uniform=True):
    if uniform:
        init_range=tf.sqrt(6.0/(n_inputs+n_outputs))
        return tf.random_uniform_initializer(-init_range, init_range)
    else:
        stddev = tf.sqrt(3.0/(n_inputs + n_outputs))
        return tf.truncated_normal_initializer(stddev=stddev)


In [3]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100
display_step =1

X = tf.placeholder(tf.float32, [None, 784])
Y = tf.placeholder(tf.float32, [None, 10])
dropout_rate = tf.placeholder("float")

W1 = tf.get_variable("W1", shape=[784, 256], initializer=xavier_init(784, 256))
W2 = tf.get_variable("W2", shape=[256, 256], initializer=xavier_init(256, 256))
W3 = tf.get_variable("W3", shape=[256, 256], initializer=xavier_init(256, 256))
W4 = tf.get_variable("W4", shape=[256, 256], initializer=xavier_init(256, 256))
W5 = tf.get_variable("W5", shape=[256,  10], initializer=xavier_init(256,  10))

B1 = tf.Variable(tf.random_normal([256]))
B2 = tf.Variable(tf.random_normal([256]))
B3 = tf.Variable(tf.random_normal([256]))
B4 = tf.Variable(tf.random_normal([256]))
B5 = tf.Variable(tf.random_normal([10]))

_L1 = tf.nn.relu(tf.add(tf.matmul(X,W1),B1))
L1  =tf.nn.dropout(_L1, dropout_rate )
_L2 = tf.nn.relu(tf.add(tf.matmul(L1,W2),B2))
L2  =tf.nn.dropout(_L2, dropout_rate)
_L3 = tf.nn.relu(tf.add(tf.matmul(L2,W3),B3))
L3  =tf.nn.dropout(_L3, dropout_rate)
_L4 = tf.nn.relu(tf.add(tf.matmul(L3,W4),B4))
L4  =tf.nn.dropout(_L4, dropout_rate)
hypothesis = tf.add(tf.matmul(L4, W5), B5)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y,logits=hypothesis))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

init = tf.global_variables_initializer()

In [4]:
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(training_epochs):
        avg_cost = 0
        total_batch = int(mnist.train.num_examples / batch_size)
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            sess.run(optimizer, feed_dict={X: batch_xs, Y: batch_ys, dropout_rate:0.7})

            avg_cost += sess.run(cost, feed_dict={X:batch_xs, Y:batch_ys, dropout_rate:0.7})/ total_batch
            
        if epoch% display_step ==0:
             print('Epoch:', '%04d' % (epoch + 1),'cost =', '{:.9f}'.format(avg_cost))

    print("Learning finished")
    
    correct_prediction = tf.equal(tf.argmax(hypothesis,1), tf.argmax(Y,1))
    accuracy =tf.reduce_mean(tf.cast(correct_prediction,"float"))
    print("Accuracy: ", accuracy.eval(feed_dict = {X: mnist.test.images, Y: mnist.test.labels, dropout_rate:1}))
    # Test the model using test sets
   
   

Epoch: 0001 cost = 0.578427750
Epoch: 0002 cost = 0.211947450
Epoch: 0003 cost = 0.161339459
Epoch: 0004 cost = 0.129860303
Epoch: 0005 cost = 0.111176618
Epoch: 0006 cost = 0.100552992
Epoch: 0007 cost = 0.089899302
Epoch: 0008 cost = 0.083034120
Epoch: 0009 cost = 0.077366428
Epoch: 0010 cost = 0.074403891
Epoch: 0011 cost = 0.068209694
Epoch: 0012 cost = 0.064344089
Epoch: 0013 cost = 0.060650985
Epoch: 0014 cost = 0.056408020
Epoch: 0015 cost = 0.053173055
Learning finished
Accuracy:  0.9802
